<a href="https://colab.research.google.com/github/gabriellaaileen/tensorflow-certification/blob/main/Question3_Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import copy, cv2,glob, shutil
import os
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import zipfile

In [2]:
class mC(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.96):
            self.model.stop_training = True
callbacks = mC()

In [3]:
from google.colab import drive

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip

--2022-07-30 03:52:41--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 172.217.194.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rps.zip’

/tmp/rps.zip        100%[===================>] 191.38M   268MB/s    in 0.7s    

2022-07-30 03:52:42 (268 MB/s) - ‘/tmp/rps.zip’ saved [200682221/200682221]



In [4]:
lz = "/tmp/rps.zip"
zip_ref = zipfile.ZipFile(lz, 'r')
zip_ref.extractall('/tmp/data')
zip_ref.close()

In [5]:
base_dir = '/tmp/rps'
data_dir = '/tmp/data/rps'
t_dir = os.path.join(base_dir,'train')
val_dir = os.path.join(base_dir,'val')
roc_dir = os.path.join(data_dir,'rock')
pa_dir = os.path.join(data_dir,'paper')
sci_dir = os.path.join(data_dir,'scissors')

In [6]:
os.mkdir(base_dir)
os.mkdir(t_dir)
os.mkdir(val_dir)

In [7]:
tr = os.path.join(t_dir, 'rock')
tp = os.path.join(t_dir, 'paper')
ts = os.path.join(t_dir, 'scissors')
vr = os.path.join(val_dir, 'rock')
vp = os.path.join(val_dir, 'paper')
vs = os.path.join(val_dir, 'scissors')

In [8]:
os.mkdir(tr)
os.mkdir(tp)
os.mkdir(ts)
os.mkdir(vr)
os.mkdir(vp)
os.mkdir(vs)

In [9]:
train_sci_dir, val_sci_dir = train_test_split(os.listdir(sci_dir), test_size = 0.30)
train_roc_dir, val_roc_dir = train_test_split(os.listdir(roc_dir), test_size = 0.30)
train_pa_dir, val_pa_dir = train_test_split(os.listdir(pa_dir), test_size = 0.30)

In [10]:
for file in train_roc_dir:
    shutil.copy(os.path.join(roc_dir, file), os.path.join(tr, file))
for file in val_roc_dir:
    shutil.copy(os.path.join(roc_dir, file), os.path.join(vr,file))
for file in train_pa_dir:
    shutil.copy(os.path.join(pa_dir,file), os.path.join(tp,file))
for file in val_pa_dir:
    shutil.copy(os.path.join(pa_dir,file), os.path.join(vp,file))

for file in train_sci_dir:
    shutil.copy(os.path.join(sci_dir,file), os.path.join(ts,file))
for file in val_sci_dir:
    shutil.copy(os.path.join(sci_dir,file), os.path.join(vs,file))

In [11]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 45,
    horizontal_flip = True,
    shear_range = 0.35,
    fill_mode = 'nearest',
)

test_datagen = ImageDataGenerator(
    rescale = 1./225,
    rotation_range = 45,
    horizontal_flip = True,
    vertical_flip = True,
    shear_range = 0.35,
    fill_mode = 'nearest'
)

In [12]:
train_generator = train_datagen.flow_from_directory(
    t_dir,
    target_size=(150,150),
    batch_size= 32,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 1764 images belonging to 3 classes.
Found 756 images belonging to 3 classes.


In [32]:
#Buatlah model dengan output layer 3
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (5,5), activation = 'relu', input_shape= (150,150,3)),
  tf.keras.layers.AvgPool2D(2,2),
  tf.keras.layers.Conv2D(128,(5,5), activation= 'relu'),
  tf.keras.layers.AvgPool2D(2,2),
  tf.keras.layers.Conv2D(256,(5,5), activation= 'relu'),
  tf.keras.layers.AvgPool2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(512, activation= 'relu'),
  tf.keras.layers.Dense(3, activation= 'softmax')
])


In [34]:
#gunakan loss categorical_crossentropy
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Adam(),
    metrics=['accuracy']
)

In [35]:
#train model
history = model.fit(
    train_generator,
    steps_per_epoch=55, 
    epochs = 42,
    validation_data = val_generator,
    validation_steps = 23, 
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/42
55/55 [==============================] - 23s 393ms/step - loss: 1.5383 - accuracy: 0.3476 - val_loss: 1.0953 - val_accuracy: 0.4226
Epoch 2/42
55/55 [==============================] - 20s 368ms/step - loss: 1.1116 - accuracy: 0.4532 - val_loss: 1.1070 - val_accuracy: 0.3723
Epoch 3/42
55/55 [==============================] - 20s 365ms/step - loss: 0.8694 - accuracy: 0.5976 - val_loss: 1.7911 - val_accuracy: 0.4918
Epoch 4/42
55/55 [==============================] - 21s 388ms/step - loss: 0.6120 - accuracy: 0.7454 - val_loss: 5.9317 - val_accuracy: 0.4837
Epoch 5/42
55/55 [==============================] - 24s 444ms/step - loss: 0.4474 - accuracy: 0.8112 - val_loss: 3.1048 - val_accuracy: 0.6182
Epoch 6/42
55/55 [==============================] - 21s 390ms/step - loss: 0.3329 - accuracy: 0.8707 - val_loss: 2.9729 - val_accuracy: 0.6209
Epoch 7/42
55/55 [==============================] - 20s 360ms/step - loss: 0.2154 - accuracy: 0.9197 - val_loss: 3.4182 - val_accuracy: 0.6427

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
%cd /content/gdrive/MyDrive/Model

/content/gdrive/MyDrive/Model


In [18]:
#save model
model.save('Model3_2.h5')